# The alfie package - demo


## Part 1. Evaluate sequences with alfie's pre-built kingdom level classifier

Alfie's primary function is as a kingdom-level taxonomic classifier for COI-5P barcode data. To accomplish this, alfie uses a deep neural network to analyze a set of input sequences and predict taxonomy. 


### Read in data 

Alfie contains a series of functions for reading and writing DNA sequence data in fasta or fastq format. These functions are found in the seqio module and their import is demonstrated below. Alfie also constains two example files that we import below using the read_fasta and read_fastq functions.

In [12]:
# import functions for fasta/fastq input and output 
from alfie.seqio import read_fasta, read_fastq, write_fasta, write_fastq

In [13]:
#import path to example file
from alfie import ex_fastq_file

#read in example file
example_fastq = read_fastq(ex_fastq_file)

#check format
example_fastq[0]

{'name': '@seq1_plantae',
 'sequence': 'ttctaggagcatgtatatctatgctaatccgaatggaattagctcaaccaggtaaccatttgcttttaggtaatcaccaagtatacaatgttttaattacagcacatgcttttttaatgattttttttatggtaatgcctgtaatgattggtggttttggtaattggttagttcctattatgataggaagtccagatatggcttttcctagactaaataacatatctttttgacttcttccaccttctttatgtttacttttagcttcttcaatggttgaagtaggtgttggaacaggatgaactgtttatcctccccttagttcgatacaaagtcattcaggcggagctgttgatttagcaatttttagcttacatttatctggagcttcatcgattttaggagctgtcaattttatttctacgattctaaatatgcgtaatcctgggcaaagcatgtatcgaatgccattatttgtttgatctatttttgtaacggca',
 'strand': '+',
 'quality': '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [14]:
#repeat the above process with a fasta file
from alfie import ex_fasta_file

example_fasta = read_fasta(ex_fasta_file)

example_fasta[0]

{'name': 'seq1_plantae',
 'sequence': 'TTCTAGGAGCATGTATATCTATGCTAATCCGAATGGAATTAGCTCAACCAGGTAACCATTTGCTTTTAGGTAATCACCAAGTATACAATGTTTTAATTACAGCACATGCTTTTTTAATGATTTTTTTTATGGTAATGCCTGTAATGATTGGTGGTTTTGGTAATTGGTTAGTTCCTATTATGATAGGAAGTCCAGATATGGCTTTTCCTAGACTAAATAACATATCTTTTTGACTTCTTCCACCTTCTTTATGTTTACTTTTAGCTTCTTCAATGGTTGAAGTAGGTGTTGGAACAGGATGAACTGTTTATCCTCCCCTTAGTTCGATACAAAGTCATTCAGGCGGAGCTGTTGATTTAGCAATTTTTAGCTTACATTTATCTGGAGCTTCATCGATTTTAGGAGCTGTCAATTTTATTTCTACGATTCTAAATATGCGTAATCCTGGGCAAAGCATGTATCGAATGCCATTATTTGTTTGATCTATTTTTGTAACGGCA'}

### Classify sequences

Once we have imported the data with the seqio module, we can use the `classify_records` function to obtain taxonomic classifications for the input sequences.   

In [5]:
# import classification function
from alfie.classify import classify_records

seq_records, predictions = classify_records(example_fasta)


The classification process is completed above in just one line of code. In the background the `classify_records` function is taking the input sequences, generating a set of input features (k-mer frequencies) for each sequence, and passing the feature sets through a neural network to obtain a prediction. 

The function yields two outputs, a list of sequence records and an array classifications. The sequence records are a list of dictonaries, like the inputs, but now with an additional 'kmer_data' entry which contains a class instance used to generate the input features for the neural network (more information on the `KmerFeatures` class is provided in the following section on training a classifier).

In [6]:
seq_records[0]

{'name': 'seq1_plantae',
 'sequence': 'TTCTAGGAGCATGTATATCTATGCTAATCCGAATGGAATTAGCTCAACCAGGTAACCATTTGCTTTTAGGTAATCACCAAGTATACAATGTTTTAATTACAGCACATGCTTTTTTAATGATTTTTTTTATGGTAATGCCTGTAATGATTGGTGGTTTTGGTAATTGGTTAGTTCCTATTATGATAGGAAGTCCAGATATGGCTTTTCCTAGACTAAATAACATATCTTTTTGACTTCTTCCACCTTCTTTATGTTTACTTTTAGCTTCTTCAATGGTTGAAGTAGGTGTTGGAACAGGATGAACTGTTTATCCTCCCCTTAGTTCGATACAAAGTCATTCAGGCGGAGCTGTTGATTTAGCAATTTTTAGCTTACATTTATCTGGAGCTTCATCGATTTTAGGAGCTGTCAATTTTATTTCTACGATTCTAAATATGCGTAATCCTGGGCAAAGCATGTATCGAATGCCATTATTTGTTTGATCTATTTTTGTAACGGCA',
 'kmer_data': <alfie.kmerseq.KmerFeatures at 0x7f462c9db990>}

The predictions returned are an encoded array of kingdom classifications.
Encodings are in alphabetical order: 
```
0 == "animalia", 1 == "bacteria", 2 == "fungi", 3 == "plantae", 4 == "protista"
```

In [7]:
predictions[:5]

array([3, 1, 4, 0, 0])

For some purposes, the names corresponding to the encoded predictions may be preferable to the numeric encodings. To obtain these use the `decode_predictions` to move from the array of numeric predictions to a list of names.  

In [8]:
from alfie.classify import decode_predictions

predicted_kingdoms = decode_predictions(predictions)
predicted_kingdoms[:5]

['plantae', 'bacteria', 'protista', 'animalia', 'animalia']

Working within python provides the freedom to manipulate the sequence records in various ways using this information. What you do with the classification information will depend on your research goal, you may wish to save a select category of sequence to a file, merge the classifications with the existing sequence ids, or carry the classifications through to additional analyses in python. Here explore a few of these possibilities.


**a.** Add the predictions into the sequence dictonaries prior to subsequent manipulation.

In [10]:
#iterate over the predictions, use enumerate to get record number
for i, p in enumerate(predicted_kingdoms):
    #call corresponding sequence record and add a kingdom entry to dictonary
    seq_records[i]['kingdom'] = p
    
#taxonomic classification now present in the sequence dict
seq_records[0]

{'name': 'seq1_plantae',
 'sequence': 'TTCTAGGAGCATGTATATCTATGCTAATCCGAATGGAATTAGCTCAACCAGGTAACCATTTGCTTTTAGGTAATCACCAAGTATACAATGTTTTAATTACAGCACATGCTTTTTTAATGATTTTTTTTATGGTAATGCCTGTAATGATTGGTGGTTTTGGTAATTGGTTAGTTCCTATTATGATAGGAAGTCCAGATATGGCTTTTCCTAGACTAAATAACATATCTTTTTGACTTCTTCCACCTTCTTTATGTTTACTTTTAGCTTCTTCAATGGTTGAAGTAGGTGTTGGAACAGGATGAACTGTTTATCCTCCCCTTAGTTCGATACAAAGTCATTCAGGCGGAGCTGTTGATTTAGCAATTTTTAGCTTACATTTATCTGGAGCTTCATCGATTTTAGGAGCTGTCAATTTTATTTCTACGATTCTAAATATGCGTAATCCTGGGCAAAGCATGTATCGAATGCCATTATTTGTTTGATCTATTTTTGTAACGGCA',
 'kmer_data': <alfie.kmerseq.KmerFeatures at 0x7f462c9db990>,
 'kingdom': 'plantae'}

**b.** Use the predictions to subset out only data from a kingdom of interest.

In [13]:
animal_sequences = []

for i, x in enumerate(predicted_kingdoms):
    if x == 'animalia':
        animal_sequences.append(seq_records[i])
        
#Note: you could avoid the transition to string classifications and 
#subset using the numberic classifications in the `predictions` array 


Below we see that the resulting list `animal_sequences` contains only the kingdom 'animalia'. 

In [14]:
animal_sequences[:2]

[{'name': 'seq4_animalia',
  'sequence': 'AATCCGGGATCATTAATTGGTGATGATCAAATTTATAATACCATTGTTACAGCTCATGCATTTATTATAATTTTTTTTATGGTTATACCAATTATAATCGGAGGATTTGGTAATTGATTAGTACCATTGATATTAGGGGCACCTGATATAGCTTTCCCACGAATAAATAATATAAGATTTTGATTACTACCCCCTTCTTTAATACTTCTAATTTCTAGTAGTATTGTAGAAAATGGAGCTGGAACTGGATGAACAGTTTACCCCCCTTTATCATCTAATATCGCCCATGGAGGAAGATCTGTTGACTTAGCTATTTTTTCATTACATTTAGCTGGTATTTCATCTATTTTAGGAGCTATTAATTTTATT',
  'kmer_data': <alfie.kmerseq.KmerFeatures at 0x7f46d4a9db10>,
  'kingdom': 'animalia'},
 {'name': 'seq5_animalia',
  'sequence': 'CAAATTTATAATACAATTGTTACAGCCCATGCTTTTATTATAATTTTCTTTATAGTAATGCCTATTATAATTGGAGGATTTGGAAATTGATTAGTACCTTTAATATTAGGAGCCCCCGATATAGCTTTCCCCCGAATAAATAATATAAGATTTTGACTTCTCCCCCCATCATTAACCCTTTTAATTTCAAGAAGAGTTGTAGAAAATGGTACTGGAACTGGATGAACAGTTTACCCCCCTTTATCATCTAATATTGCTCATAGAGGAAGATCTGTTGATTTATCTATTTTTTCCCTTCATTTAGCTGGAATTTCTTCTATTTTAGGAGCAATTAATTTTATTACAACTATTATTAATATACGATTAAATAATATAACATTTGATCAATTACCTTTATTTGTATGATCTGTTGGAATTACAGCTCTTCTTCTTCTTCTTTCTCTTCCTGTTTTAGC

**c.** Writing to file

Once processing and analyses are completed, we may wish to save our sequence records to a new output file. Sequences, or lists of sequences in dictonary format can be written to output files if they possess the proper set of keys ('name' and 'sequence' for fasta, 'name', 'sequence', 'strand', and 'quality'). Additional keys will be ignored when writing the output.

In this demonstration, we take the animal sequences we isolated from the input (**b.**) and write them to a new fasta file.

In [ ]:
# if you uncomment and run the following line, it will make an output file in your current working directory
#write_fasta( animal_sequences, 'animalia_example_output.fasta')

Thats all there is to deploying the alfie package as a kingdom level classifier from within Python. The kingdom level classification provides an efficient means of separating DNA sequences from a target kingdom from the large amount of off-target noise that can exist within a metabarcoding or environmental DNA data set.

Next, we explore how the functionality of alfie can be customized to allow for isolation of target sequences on finer taxonomic scales.


## Part 2. Train and test a custom, alignment-free taxonomic classifier

In addition to using alfie as a kingdom level classifier, alfie's helper functions can be used to train a custom DNA barcode classification model. Custom model construction will allow for the general functionality of alfie (as a kingdom-level classifier) to be extended and specalized. Some common applications of this customization may be the training of a classifier for a sub-group of interest (i.e. an group classifier, which is demonstrated here for the phylum annelida), or training a binary classifier to isolate barcodes from a specific taxonomic group (i.e. a classifier that says whether an input sequence is or is not from a teleost fish).

The following demonstration will show how to train a custom binary or multiclass neural network through a combination of alfie, scikit learn, and tensorflow. Note this process is a little more involved than the default implementation of alfie. This demo assumes an understanding of the basics of data science and machine learning in Python. If you're not yet comfortable with those topics, I would recommend the book [Hands-on Machine Learning with Scikit-Learn and TensorFlow](https://github.com/ageron/handson-ml2) as a good starting point.

It is also important to note that your mileage with a design and deployment of custom classifier will vary with the quality of the training data you use. A few thousand labelled sequences at a minimum is recommended. If you're looking to acquire COI training data, consider: [the BOLD data systems website](http://www.boldsystems.org/index.php/Login/page?destination=MAS_Management_UserConsole), or [subsetting the data used in training the original alfie neural network](https://github.com/CNuge/data-alfie). 

If you want training data for other barcodes or genes have a look at [NCBI](https://www.ncbi.nlm.nih.gov)(warning: data mining and cleaning required), or other online barcode data sources such as the [PLANiTS dataset](https://github.com/apallavicini/PLANiTS). Another good source of barcode data is [Dr. Teresita Porter's GitHub page](https://github.com/terrimporter), which contains trained RDP Classifiers (and labelled training data!) for rbcL, 18S, ITS, and other barcodes.


In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelBinarizer



In [2]:
from alfie.kmerseq import KmerFeatures
from alfie.training import stratified_taxon_split, sample_seq, process_sequences, alfie_dnn_default

### loading the demo data

The demo data can be found in the [alfie GitHub repository](https://github.com/CNuge/alfie/tree/master/alfie/data). The relative import below assumes you have downloaded the alfie repository from gihub and that your working directory is: `alfie/example`.

In [3]:
data = pd.read_csv('../alfie/data/alfie_small_train_example.tsv', sep = '\t')

In [4]:
data.head()

,processid,sequence,phylum,class,order,family,genus
0,GAHAP309-13,accttatactttattctgggcgtatgagcaggaatattgggtgcag...,Annelida,Clitellata,Enchytraeida,Enchytraeidae,Grania
1,GAHAP2002-14,accctatatttcattctcggagtttgagctggcatagtaggtgccg...,Annelida,Clitellata,Haplotaxida,Lumbricidae,Aporrectodea
2,GBAN15302-19,actctatacttaatttttggtatt-gagccggtatagtaggaacag...,Annelida,Clitellata,Haplotaxida,Naididae,Ainudrilus
3,GBAN11905-19,acactatattttattttaggaatttgagctggaataattggagcag...,Annelida,Clitellata,Crassiclitellata,Megascolecidae,Metaphire
4,GBAN15299-19,acattatacctaattta-ggtgtatgagccggaatagttggaacag...,Annelida,Clitellata,Haplotaxida,Naididae,Ainudrilus


For similicity, the demo is conducted with 10,000 sequences from the phylum Annelida, which has only two classes. We will train a model to predict the class for Annelida sequences. 


In [5]:
data['class'].value_counts()

Clitellata    6187
Polychaeta    3813
Name: class, dtype: int64

### Conducting a train/test split


The aflie function `stratified_taxon_split` can be used to split a dataframe in a stratified fashion based on the taxnomic data in a column. This ensures that each taxonomic group is evenly represented in the training and test data.

In [6]:
train, test = stratified_taxon_split(data, class_col = 'class', test_size = 0.3, )

Conducting train/test split, split evenly by: class


We can call some summary functions on the output dataframes to verify the even split of the data.

In [7]:
test.shape

(3000, 7)

In [8]:
print("train data shape:",train.shape)
print(train['class'].value_counts())
print("\n")

print("test data shape:", test.shape)
print(test['class'].value_counts())


train data shape: (7000, 7)
Clitellata    4331
Polychaeta    2669
Name: class, dtype: int64


test data shape: (3000, 7)
Clitellata    1856
Polychaeta    1144
Name: class, dtype: int64


In [ ]:
test['class'].value_counts()

### Encoding the response data


In [9]:
print("encoding y arrays")
#encode the y labels
y_train_raw =  train['class']
y_test_raw = test['class']

tax_encoder = LabelBinarizer()

y_train = tax_encoder.fit_transform(y_train_raw)
y_test = tax_encoder.transform(y_test_raw)



encoding y arrays


In [10]:
print(y_train.shape)
print(y_test.shape)


(7000, 1)
(3000, 1)


In [ ]:
"""
# uncomment these lines to save the arrays to files on your computer
print("saving y to files")
np.save('example_y_train.npy', y_train)
np.save('example_y_test.npy', y_test)
"""

### Encoding the predictor data


#### kmer features

Alfie contains a custom python class called KmerFeatures, which intakes a DNA sequence and generates kmer count and kmer frequency data. This class is used in the background by the `classify_records` function to generate the features for neural network prediction. 

Here we will use it to take our data and generate the feature sets for model training. The class takes an id and a DNA sequence, by default it will count 4mer frequencies.

In [35]:
#uncomment to look at the docs
?KmerFeatures

In [20]:
x1 = KmerFeatures(name = train['processid'][0]  , sequence = train['sequence'][0])
x1

Upon initiation, the KmerFeatures class instance will generate a kmer count dictonary, where the keys are all the nucleotide permutations for size 'k'. The class then iterates through the input sequence and counts the occurances of each kmer. Any occurances of nucleotides other than A, T, G, and C will cause the encompassing kmer to be ignored.

After initiation, the kmer keys and values can be accessed like a regular python dictonary.

In [41]:
x1.keys()[:10]

['AAAA',
 'AAAC',
 'AAAG',
 'AAAT',
 'AACA',
 'AACC',
 'AACG',
 'AACT',
 'AAGA',
 'AAGC']

[Note/digression: later we rely on the fact keys are from an alphabetically ordered dict (the default for python 3.6 or newer). Alfie sorts the dict items by key before returning them to be safe. But this is a PSA that its 2020 and time to update if you're on 3.5 or earlier!](https://twitter.com/raymondh/status/773978885092323328)

In [42]:
x1.values()[:10]

[2, 3, 1, 2, 2, 3, 1, 4, 3, 1]

In [43]:
x1.items()[:10]

[('AAAA', 2),
 ('AAAC', 3),
 ('AAAG', 1),
 ('AAAT', 2),
 ('AACA', 2),
 ('AACC', 3),
 ('AACG', 1),
 ('AACT', 4),
 ('AAGA', 3),
 ('AAGC', 1)]

So that the training data is not biased by the overall size of the sequences, it is best to train the models using the kmer frequencies (count/total), which the KmerFeatures class also provides for us.

In [46]:
x1.freq_values()[:10]

array([0.0030722 , 0.00460829, 0.0015361 , 0.0030722 , 0.0030722 ,
       0.00460829, 0.0015361 , 0.00614439, 0.00460829, 0.0015361 ,
       0.00460829, 0.        , 0.01228879, 0.00460829, 0.        ,
       0.01075269, 0.0030722 , 0.0015361 , 0.00614439, 0.00460829,
       0.00460829, 0.0030722 , 0.0015361 , 0.00614439, 0.0030722 ,
       0.        , 0.        , 0.        , 0.00614439, 0.        ,
       0.0030722 , 0.01228879, 0.0015361 , 0.0030722 , 0.0015361 ,
       0.00460829, 0.01228879, 0.00460829, 0.0015361 , 0.00460829,
       0.01075269, 0.        , 0.        , 0.00768049, 0.00614439,
       0.        , 0.        , 0.00460829, 0.00768049, 0.00768049,
       0.0015361 , 0.00921659, 0.0030722 , 0.00460829, 0.0015361 ,
       0.00614439, 0.0030722 , 0.00460829, 0.        , 0.        ,
       0.01689708, 0.00614439, 0.00768049, 0.01689708, 0.        ,
       0.00460829, 0.0030722 , 0.0030722 , 0.00460829, 0.0015361 ,
       0.0015361 , 0.0030722 , 0.        , 0.00614439, 0.01075

For machine learning purposes, the KmerFeatures class also outputs the kmer dictonary keys and value frequencies as numpy arrays.

In [44]:
x1.labels[:5] #only showing first 5 

array(['AAAA', 'AAAC', 'AAAG', 'AAAT', 'AACA'], dtype='<U4')

In [45]:
x1.kmer_freqs[:5] #just static method version of x1.freq_values()

array([0.0030722 , 0.00460829, 0.0015361 , 0.0030722 , 0.0030722 ])

By passing the optional kmers argument, we can overrule the default kmer size of 4 and specify a custom kmer dictonary size. 

In [36]:
# count 5mers 
x5mer = KmerFeatures(name = train['processid'][0]  , sequence = train['sequence'][0], kmers = 5)

print(x5mer.items()[:5])

#generate 5mer and 1mer frequencies
x1_mer = KmerFeatures(name = train['processid'][0]  , sequence = train['sequence'][0], kmers = 1)
print(x1_mer.items())


TypeError: 'int' object is not iterable

#### random subsampling

The kmer encoding of the predictor data above utilizes the whole barcode records. Often in analysis of metabarcode or eDNA data we are not dealing with complete barcode sequences, but rather short sequences (from primer-defined barcode sub-sections in the case of metabarcoding data, or undefined sub-sections in the case of metagenomics data). 

To train our classification model on data that more closely resembles real world sequences, we can use the alfie's `sample_seq` function to randomly subsample the training sequences.

In [ ]:


#demonstrate on a single sequence - i.e. row 1 of the train

sub_seq = sample_seq(train['sequence'][0])



can be used for upsampling as well

In [ ]:
sub_seq = sample_seq(train['sequence'][0], n = 10)


#### batch processing

Here the data are processed with the defaults kmer size (`k = [4]`) and a single subsample per sequence in the input dataframe (`n = 1`).

In [ ]:
train_kmer_data = process_sequences(train, label_col = 'class')
test_kmer_data = process_sequences(train, label_col = 'class')

In [ ]:
#scramble the order if you upsampled the data
from alfie.training import shuffle_unison

?shuffle_unison

The outputs dictonaries of the process_sequences function can be easily turned into numpy arrays, which are compatible inputs for most machine learning algorithms.

In [ ]:
print("building X arrays")
X_train = np.array(train_samples['data'])
X_test = np.array(test_samples['data'])


In [ ]:
"""
# uncomment these lines to save the arrays to your computer
print("saving X to files")
np.save('../train/fouronly_single_final_kingdom_X_train.npy', X_train)
np.save('../test/fouronly_single_final_kingdom_X_test.npy', X_test)
"""

#### training the neural network